# Introduction
Continuing on the more advanced topics of Silicon, today we're going to play around with structs. We will start with fairly simple use-cases and will slowly work our way to more advanced uses, including introducing behaviors.

# Structs?
Yes, structs. The good old grab-bag of containers we know and love from other languages, such as C, VHDL (they call it `record`) and even SystemVerilog. Interestingly enough Python, our host language doesn't natively support structs, though [`namedtuple`](https://docs.python.org/3/library/collections.html#collections.namedtuple)s come fairly cose.

Either way, the concept is the same and is a simple one: take a few other types and package them together to represent data that is usually used together. Simple examples include the first and last-name of a person, the IP address and the port for a connection or the red, green and blue values for a color.

Structs are one of the most powerful ways to extend the type-system of a language, so naturally Silicon has support for them as well

# Structs in Silicon


## The prerequisites
Before we can start, let's set up Silicon

In [1]:
# This is needed for local setups only so Python finds silicon 
import sys
sys.path.append("..")

from silicon import *

## The Example
What we're going to build is a simple module, that takes two pixels (defined by their RGB values), an alpha (opacity value) and outputs a mix of the two pixels. If alpha is 0, the output is going to be the second pixel, if alpha is 255, the output is the first pixel. Intermediate alpha values will give a weighted average of the two pixels. We are going to represent each color channel by an 8-bit value. For simplicity, we'll assume no [gamma](https://en.wikipedia.org/wiki/Gamma_correction), that is to say that numerical values encode linear brightness.

If we scaled all values in question to represent values between 0 and 1, the math is rather simple: `outp = in1 * alpha + in2 * (1-alpha)`.

Let's start by creating a monochrome version of the module!


In [2]:
class AlphaBlender(Module):
    in1 = Input(Unsigned(8))
    in2 = Input(Unsigned(8))
    alpha = Input(Unsigned(8))
    outp = Output(Unsigned(8))
    
    def body(self):
        pix1 = self.in1 * self.alpha
        pix2 = self.in2 * (255-self.alpha)
        self.outp = (pix1 + pix2 + 127)[15:8]


We're using fractional math here, so we'll have to use the upper-half of the result. We're also adding `127` to the result to get some simple rounding behavior.

Let's generate the RTL for it to see what we get:


In [3]:
def gen_rtl(top):
    netlist = elaborate(top)
    rtl = StrStream()
    netlist.generate(netlist, SystemVerilog(rtl))
    print(rtl)

gen_rtl(AlphaBlender())

////////////////////////////////////////////////////////////////////////////////
// AlphaBlender
////////////////////////////////////////////////////////////////////////////////
module AlphaBlender (
	input logic [7:0] in1,
	input logic [7:0] in2,
	input logic [7:0] alpha,
	output logic [7:0] outp
);

	logic [15:0] pix1;
	logic [15:0] pix2;

	assign pix1 = in1 * alpha;
	assign pix2 = in2 * (8'hff - alpha);
	assign outp = (pix1 + pix2 + 7'h7f)[15:8];

endmodule





That was simple enough, but let's now try to do this in color!

First we'll have to introduce a data structure to represent our pixels:

In [4]:
class Pixel(Struct):
    r = Unsigned(8)
    g = Unsigned(8)
    b = Unsigned(8)

Notice, that we created *types* as opposed to inputs or outputs within this construct. Other then that, it looks fairly similar to a Module definition.

The description is rather intuitive, but just to be clear, let's spell it out: a struct class in Silicon is a child of the `Struct` class. The simplest form of creating such a type is to inherit from `Struct` and in the body of the class, list all the members in the format of `<name> = <type>`.

Now, let's incorporate this type into our module:

In [5]:
class AlphaBlender(Module):
    in1 = Input(Pixel())
    in2 = Input(Pixel())
    alpha = Input(Unsigned(8))
    outp = Output(Pixel())
    
    def body(self):
        #pix1 = self.in1 * self.alpha
        #pix2 = self.in2 * (255-self.alpha)
        #self.outp = (pix1 + pix2 + 127)[15:8]
        pass

I've commented out all the body of the code because we immediately run into a problem: we can't simply multiply a `Pixel` with a `Number`. We also can't add them together or take a slice out of them. We have to do that member by member. So, unfortunately we would have to triplicate the code, but we can do one better: we can introduce a function and call that three times: 

In [6]:
class AlphaBlender(Module):
    in1 = Input(Pixel())
    in2 = Input(Pixel())
    alpha = Input(Unsigned(8))
    outp = Output(Pixel())
    
    def body(self):
        
        def blend_mono(in1, in2):
            pix1 = in1 * self.alpha
            pix2 = in2 * (255-self.alpha)
            return (pix1 + pix2 + 127)[15:8]
        self.outp.r = blend_mono(self.in1.r, self.in2.r)
        self.outp.g = blend_mono(self.in1.g, self.in2.g)
        self.outp.b = blend_mono(self.in1.b, self.in2.b)


Let's see what we've cooked up!

In [7]:
gen_rtl(AlphaBlender())

////////////////////////////////////////////////////////////////////////////////
// AlphaBlender
////////////////////////////////////////////////////////////////////////////////
module AlphaBlender (
	input logic [7:0] in1_b,
	input logic [7:0] in1_g,
	input logic [7:0] in1_r,

	input logic [7:0] in2_b,
	input logic [7:0] in2_g,
	input logic [7:0] in2_r,

	input logic [7:0] alpha,
	output logic [7:0] outp_b,
	output logic [7:0] outp_g,
	output logic [7:0] outp_r
);

	assign outp_r = (in1_r * alpha + in2_r * (8'hff - alpha) + 7'h7f)[15:8];
	assign outp_g = (in1_g * alpha + in2_g * (8'hff - alpha) + 7'h7f)[15:8];
	assign outp_b = (in1_b * alpha + in2_b * (8'hff - alpha) + 7'h7f)[15:8];

endmodule





A few notes here:

1. As expected, `Struct`-members are accessed with the usual `.`-notation.
2. `Struct` members can be individually assigned. It also means that a struct can be partially assigned, some members having a source, some others don't.
3. We can effectively use functions to abstract common logic out. This is a bit trickier than it sounds: remember, that we're not actually executing anything, just build a data-flow graph. So, the inputs to the function `blend_mono` are not integers, but `Port`s and its output is also a `Port`. These ports get bound to whatever the input arguments and the return values are of the function at the call-site. If you call a function three times, its 'guts' gets instantiated three times. Which is of course what we wanted in this case.
4. The generated RTL is pretty clean, but we needed to add the  attribute. This might change in the future, but the point of the attribute is to prevent the RTL generator to attempt to inject the names used within the function (`in1`, `in2`, `pix1`, `pix2`) into the calling namespace. If you get this wrong, the generated RTL would still be valid, but would look much uglier.
5. The members of `Pixel` got elevated to the module level during RTL generation: they appear as independent wires, though their name still suggests they belong together. There are many reasons for this design choice (after all SystemVerilog has a structs), and it basically boils down to limitations with SystemVerilog interfaces.

# Composition
Let's think about a more complex example! Let's say, we wanted to incorporate this alpha blender module into a pixel pipeline. What is important to know about most pixel stream sources (cameras for example) is that they don't output a pixel in every clock cycle. They have [blanking periods](https://en.wikipedia.org/wiki/Blanking_(video)), which originally was needed for analog CRT tubes, but the implementation stuck. The actual details are much more complex, but for this simple exercise, let's just assume that we wanted to include a `valid` bit with every input pixel. We will require (assume) that both inputs are valid or invalid at the same time and our output will be valid if both inputs are valid.

Now, we could simply add this extra bit to the `Pixel` struct, but we can try something else:

In [8]:
class ValidPixel(Struct):
    pixel = Pixel()
    valid = logic

Notice, how `pixel` above is an *instance* of the `Pixel` class!

I'll be the first one to concede, this example is a bit contrived, but let me bare with me here! The point is that we've create a struct that contains a struct. How would our blender change?

In [9]:
class AlphaBlender(Module):
    in1 = Input(ValidPixel())
    in2 = Input(ValidPixel())
    alpha = Input(Unsigned(8))
    outp = Output(ValidPixel())
    error = Output(logic)
    
    def body(self):
        
        def blend_mono(in1, in2):
            pix1 = in1 * self.alpha
            pix2 = in2 * (255-self.alpha)
            return (pix1 + pix2 + 127)[15:8]
        self.outp.pixel.r = blend_mono(self.in1.pixel.r, self.in2.pixel.r)
        self.outp.pixel.g = blend_mono(self.in1.pixel.g, self.in2.pixel.g)
        self.outp.pixel.b = blend_mono(self.in1.pixel.b, self.in2.pixel.b)
        self.outp.valid = self.in1.valid & self.in2.valid
        self.error = self.in1.valid ^ self.in2.valid

gen_rtl(AlphaBlender())

////////////////////////////////////////////////////////////////////////////////
// AlphaBlender
////////////////////////////////////////////////////////////////////////////////
module AlphaBlender (
	input logic [7:0] in1_pixel_b,
	input logic [7:0] in1_pixel_g,
	input logic [7:0] in1_pixel_r,
	input logic in1_valid,

	input logic [7:0] in2_pixel_b,
	input logic [7:0] in2_pixel_g,
	input logic [7:0] in2_pixel_r,
	input logic in2_valid,

	input logic [7:0] alpha,
	output logic [7:0] outp_pixel_b,
	output logic [7:0] outp_pixel_g,
	output logic [7:0] outp_pixel_r,
	output logic outp_valid,

	output logic error
);

	assign outp_pixel_r = (in1_pixel_r * alpha + in2_pixel_r * (8'hff - alpha) + 7'h7f)[15:8];
	assign outp_pixel_g = (in1_pixel_g * alpha + in2_pixel_g * (8'hff - alpha) + 7'h7f)[15:8];
	assign outp_pixel_b = (in1_pixel_b * alpha + in2_pixel_b * (8'hff - alpha) + 7'h7f)[15:8];
	assign outp_valid = in1_valid & in2_valid;
	assign error = in1_valid ^ in2_valid;

endmodule





# Inheritance

Another way to express the same problem, is to define an descendant class, that inherits from our original `Pixel` struct and introduces a new member:

In [10]:
class Pixel(Pixel):
    valid = logic

With that definition our blender would look like this:

In [11]:
class AlphaBlender(Module):
    in1 = Input(Pixel())
    in2 = Input(Pixel())
    alpha = Input(Unsigned(8))
    outp = Output(Pixel())
    error = Output(logic)
    
    def body(self):
        
        def blend_mono(in1, in2):
            pix1 = in1 * self.alpha
            pix2 = in2 * (255-self.alpha)
            return (pix1 + pix2 + 127)[15:8]
        self.outp.r = blend_mono(self.in1.r, self.in2.r)
        self.outp.g = blend_mono(self.in1.g, self.in2.g)
        self.outp.b = blend_mono(self.in1.b, self.in2.b)
        self.outp.valid = self.in1.valid & self.in2.valid
        self.error = self.in1.valid ^ self.in2.valid

gen_rtl(AlphaBlender())

////////////////////////////////////////////////////////////////////////////////
// AlphaBlender
////////////////////////////////////////////////////////////////////////////////
module AlphaBlender (
	input logic [7:0] in1_b,
	input logic [7:0] in1_g,
	input logic [7:0] in1_r,
	input logic in1_valid,

	input logic [7:0] in2_b,
	input logic [7:0] in2_g,
	input logic [7:0] in2_r,
	input logic in2_valid,

	input logic [7:0] alpha,
	output logic [7:0] outp_b,
	output logic [7:0] outp_g,
	output logic [7:0] outp_r,
	output logic outp_valid,

	output logic error
);

	assign outp_r = (in1_r * alpha + in2_r * (8'hff - alpha) + 7'h7f)[15:8];
	assign outp_g = (in1_g * alpha + in2_g * (8'hff - alpha) + 7'h7f)[15:8];
	assign outp_b = (in1_b * alpha + in2_b * (8'hff - alpha) + 7'h7f)[15:8];
	assign outp_valid = in1_valid & in2_valid;
	assign error = in1_valid ^ in2_valid;

endmodule





# Generic structs
What if we wanted to generalize our `Pixel`s to work with arbitrary number of bits? We would need to somehow add a parameter (the bit-depth) to the struct definition. Luckily, this is rather easy to do. All we need to do is - instead of statically creating the members as we've done so far - to create the members of the struct dynamically:

In [12]:
class Pixel(Struct):
    def __init__(self, length: int):
        super().__init__()
        self.add_member("r", Unsigned(length))
        self.add_member("g", Unsigned(length))
        self.add_member("b", Unsigned(length))

Now, our blender code (the one without the valid handling) would change to this:

In [13]:
class AlphaBlender(Module):
    pixel_width = 12

    in1 = Input(Pixel(pixel_width))
    in2 = Input(Pixel(pixel_width))
    alpha = Input(Unsigned(8))
    outp = Output(Pixel(pixel_width))
    
    def body(self):
        
        def blend_mono(in1, in2):
            pix1 = in1 * self.alpha
            pix2 = in2 * (255-self.alpha)
            top = AlphaBlender.pixel_width + 8 - 1
            return (pix1 + pix2 + 127)[top:8]
        self.outp.r = blend_mono(self.in1.r, self.in2.r)
        self.outp.g = blend_mono(self.in1.g, self.in2.g)
        self.outp.b = blend_mono(self.in1.b, self.in2.b)
        
gen_rtl(AlphaBlender())

////////////////////////////////////////////////////////////////////////////////
// AlphaBlender
////////////////////////////////////////////////////////////////////////////////
module AlphaBlender (
	input logic [11:0] in1_r,
	input logic [11:0] in1_g,
	input logic [11:0] in1_b,

	input logic [11:0] in2_r,
	input logic [11:0] in2_g,
	input logic [11:0] in2_b,

	input logic [7:0] alpha,
	output logic [11:0] outp_r,
	output logic [11:0] outp_g,
	output logic [11:0] outp_b
);

	assign outp_r = (in1_r * alpha + in2_r * (8'hff - alpha) + 7'h7f)[19:8];
	assign outp_g = (in1_g * alpha + in2_g * (8'hff - alpha) + 7'h7f)[19:8];
	assign outp_b = (in1_b * alpha + in2_b * (8'hff - alpha) + 7'h7f)[19:8];

endmodule





# Behaviors

Well, it works, but it's rather ugly. The problem is that our blender code now need to know about the number of bits in each pixel. In the code above, we've saved it off into a class-level constant, but that's not the nicest thing to do. After all, Pixel should already have this information, we should not need to store it twice. (Indeed it does and there is a way to get it, but that makes the example only slightly more generic and slightly less readable.)

Wouldn't it be nice, if we could somehow package our blender code *with* `Pixel`, so all the necessary information travels together? Then all the ugliness of figuring out the slicing of the result of the blending would be hidden behind seom facade and we would not need to worry about it.

That's what we would naturally do in an object-oriented language. But wait! Python *is* an object-oriented language. So, can we take advantage of it? Of course we can:

In [14]:
class Pixel(Struct):
    def __init__(self, length: int):
        super().__init__()
        self.length = length
        self.add_member("r", Unsigned(length))
        self.add_member("g", Unsigned(length))
        self.add_member("b", Unsigned(length))
    @behavior
    
    def blend(junction, other, alpha):
        
        def blend_mono(in1, in2):
            pix1 = in1 * alpha
            pix2 = in2 * (255-alpha)
            top = junction.get_net_type().length + 8 - 1
            return (pix1 + pix2 + 127)[top:8]
        result = Wire(Pixel(junction.get_net_type().length))
        result.r = blend_mono(junction.r, other.r)
        result.g = blend_mono(junction.g, other.g)
        result.b = blend_mono(junction.b, other.b)
        return result

class AlphaBlender(Module):
    pixel_width = 12

    in1 = Input(Pixel(pixel_width))
    in2 = Input(Pixel(pixel_width))
    alpha = Input(Unsigned(8))
    outp = Output(Pixel(pixel_width))
    
    def body(self):
        self.outp = self.in1.blend(self.in2, self.alpha)
        
gen_rtl(AlphaBlender())

////////////////////////////////////////////////////////////////////////////////
// AlphaBlender
////////////////////////////////////////////////////////////////////////////////
module AlphaBlender (
	input logic [11:0] in1_r,
	input logic [11:0] in1_g,
	input logic [11:0] in1_b,

	input logic [11:0] in2_r,
	input logic [11:0] in2_g,
	input logic [11:0] in2_b,

	input logic [7:0] alpha,
	output logic [11:0] outp_r,
	output logic [11:0] outp_g,
	output logic [11:0] outp_b
);

	assign outp_r = (in1_r * alpha + in2_r * (8'hff - alpha) + 7'h7f)[19:8];
	assign outp_g = (in1_g * alpha + in2_g * (8'hff - alpha) + 7'h7f)[19:8];
	assign outp_b = (in1_b * alpha + in2_b * (8'hff - alpha) + 7'h7f)[19:8];
endmodule





There are a few things to unpack here.

1. The `blend` method appears on the `Pixel` object, but it's not actually a method of it. What happens under the hood is that the `@behavior` decorator takes this method and *injects* it to every `Junction` instance instead. So, when it's called, the first parameter is not the net type object, but the junction itself.
2. The order of the decorators is important: if you flip `@behavior` and ``, things break.
3. Since inside `blend` we don't have direct access to the type object (we don't have a `self`), we have to fish it out the hard way from the junction, by calling it's `get_net_type()` method.
4. The `result` local will have to be an actual `Wire`, not just a Pixel. This is the consequence of us building a data-flow graph, instead of executing everything. The return value of `blend` will get bound to whatever the caller decides to bind it to as the return value. In our case that's AlphaBlenders' outp port.

# Parting words
We've gone through what `Struct`s can offer you. They provide an abstraction data-type, that collects related values into a single entity. They provide member access, composition and inheritance as basic code organization features. With a little effort, one can introduce generic structs and with some more, one can imbue object-oriented features to them, using `@behaviors`.

# TODO:
1. Show example of simulation.
2. Can we maybe simplify behaviors to the point where both the junction and the type is available? Feels 'unclean' to have it as a method, yet not have a 'self'.